# MFPT Optimisation Benchmarking

In [2]:
from util_old import Markov_mfpt_calc_jjhunter
from scipy.sparse import random,linalg
from scipy import eye,diag
from scipy.linalg import inv
import config

In [54]:
MM_sparse = MM_sparse.astype(dtype = np.float64)

In [4]:
MM_sparse.toarray() ### run below code to obtain this matrix

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float128)

#### Testing MFPT calc functions:

In [3]:
M = random(10000,10000,density = 0.1)

In [3]:
M.toarray()

array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 8.35997066e-01, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [6.42530186e-01, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 6.75371364e-04, 4.01847252e-01],
       ...,
       [3.84817057e-01, 0.00000000e+00, 0.00000000e+00, ...,
        5.31113579e-02, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 5.87924133e-01, 0.00000000e+00, ...,
        9.02832473e-01, 0.00000000e+00, 0.00000000e+00],
       [9.25292982e-01, 0.00000000e+00, 0.00000000e+00, ...,
        5.60479910e-01, 0.00000000e+00, 1.12101166e-01]])

In [4]:
from util_old import compute_free_energy,compute_fre_energy_sparse, Markov_mfpt_calc_sparse,Markov_mfpt_calc_jjhunter, compute_free_energy_power_method_sparse, Markov_mfpt_calc
import numpy as np

In [5]:
MM_dense = M.toarray()
for i in range(len(MM_dense)):
    sum = np.sum(MM_dense[i])
    MM_dense[i] = MM_dense[i]/sum

In [6]:
MM_SP = config.csr_matrix(MM_dense)
MM_dense

array([[0.        , 0.        , 0.00063255, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.00165816,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.00191483, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.00081742],
       [0.00142593, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [7]:
%%time
peq = compute_free_energy(MM_dense.T)[0]

CPU times: user 32min 3s, sys: 22min 47s, total: 54min 51s
Wall time: 4min 16s


In [8]:
%%time
peq_SP = compute_fre_energy_sparse(MM_SP.T)[0] ### therefore both functions produce the same result

CPU times: user 14min 38s, sys: 38min 57s, total: 53min 36s
Wall time: 3min 30s


In [9]:
%%time
MFPT1 = Markov_mfpt_calc(peq.T,MM_dense) 

CPU times: user 3min 6s, sys: 38.7 s, total: 3min 45s
Wall time: 1min 15s


In [14]:
%%time
MFPT2 = Markov_mfpt_calc_sparse(peq.T,MM_SP)

Not enough memory to perform factorization.


MemoryError: 

In [21]:
M = random(5000,5000,density = 0.1)
MM_dense = M.toarray()
for i in range(len(MM_dense)):
    sum = np.sum(MM_dense[i])
    MM_dense[i] = MM_dense[i]/sum
MM_SP = config.csr_matrix(MM_dense)
peq = compute_free_energy(MM_dense.T)[0]


In [22]:
%%time
MFPT1 = Markov_mfpt_calc(peq.T,MM_dense) 

CPU times: user 29.9 s, sys: 13.2 s, total: 43.1 s
Wall time: 15.2 s


In [23]:
%%time
MFPT2 = Markov_mfpt_calc_sparse(peq.T,MM_SP)

CPU times: user 3min 16s, sys: 4min 19s, total: 7min 36s
Wall time: 1min 46s


In [29]:
M = random(10,10,density = 0.7)
MM_dense = M.toarray()
for i in range(len(MM_dense)):
    sum = np.sum(MM_dense[i])
    MM_dense[i] = MM_dense[i]/sum
MM_SP = config.csr_matrix(MM_dense)
peq = compute_free_energy(MM_dense.T)[0]


In [32]:
row_sum = []
column_sum = []
for i in range(len(MM_dense)):
    row_sum.append(np.sum(MM_dense[i]))
    column_sum.append(np.sum(MM_dense[:,i]))

In [34]:
row_sum,column_sum

([0.9999999999999999,
  0.9999999999999999,
  0.9999999999999999,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  0.9999999999999998],
 [0.7988900856803383,
  1.2968358539453484,
  0.5594353852890691,
  1.503815846027901,
  1.1825253853781035,
  1.0216061439648225,
  1.1298221699246882,
  0.640323955473453,
  0.7126881848606981,
  1.1540569894555774])

#### MFPT Calc time comparison

It can be seen that for a Transition Matrix of size 100 with density 0.1 the sparse matrix mean first passage time calculation is quicker. It can also be seen that for a 10000 by 10000 transition matrix with the same density it is even faster with the total time being a 5th. It should be noted that the less sparse the matrix is the less the benefits from the sparse computation.

I will now test the try and optim functions independently.

In [26]:
MM_dense

array([[0.01161207, 0.        , 0.27468903, 0.        , 0.        ,
        0.09490362, 0.        , 0.        , 0.24908483, 0.36971044],
       [0.        , 0.        , 0.32665021, 0.        , 0.02819445,
        0.14733389, 0.        , 0.19554049, 0.        , 0.30228096],
       [0.18976329, 0.        , 0.15176067, 0.2148473 , 0.        ,
        0.14468669, 0.1943436 , 0.10459846, 0.        , 0.        ],
       [0.        , 0.31517317, 0.        , 0.        , 0.24026667,
        0.31259483, 0.        , 0.13196533, 0.        , 0.        ],
       [0.        , 0.0466464 , 0.        , 0.46840608, 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.48494752],
       [0.        , 0.        , 0.        , 0.        , 0.5883228 ,
        0.        , 0.        , 0.4116772 , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.27787598, 0.        ,
        0.25215899, 0.        , 0.        , 0.0884809 , 0.38148413],
       [0.07905768, 0.        , 0.       

It is evident above that the dense MFPT calculation is quicker. This can b explained by the dense nature of the output matrix, and the inefficient conditional logic which is difficult to vectorise which is a result of the manually input large values for transitioned observed as impossible. I will now use start testing the JJ_huunter algorithm.

In [ ]:
def jjhunter(P): ### correct version
    P = np.array(P, dtype=np.float64)  # Using double precision
    m = len(P)
    I = np.eye(m, dtype=np.float64)
    e1 = np.ones((m, 1), dtype=np.float64)
    E = np.ones((m, m), dtype=np.float64)
    PPD = P.copy()
    AASD = np.zeros((m, m), dtype=np.float64)
    SD = np.zeros(m)

    for n in range(m-1, 0, -1):
        #print(n,PPD[n, :n])
        SD[n] = np.sum(PPD[n, :n])
        #print(SD)
        for i in range(n):
            for j in range(n):
                AASD[i, j] = PPD[i, n] * PPD[n, j] / SD[n]
                PPD[i, j] += AASD[i, j]
    #print(AASD,'\n', PPD)

    rD = np.zeros(m, dtype=np.float64)
    rD[0] = 1
    for n in range(1, m):
        for i in range(n):
            rD[n] += rD[i] * PPD[i, n] / SD[n]

    TOTD = np.sum(rD)
    pit_1D = rD / TOTD
    PiD = e1 @ pit_1D.reshape(1, -1)
    ZD = np.linalg.inv(I - P + PiD)
    DD = np.linalg.inv(np.diag(np.diag(PiD)))
    MD = (I - ZD + E @ np.diag(np.diag(ZD))) @ DD
    return MD

In [48]:
MFPT = jjhunter(MM_dense)

In [18]:
#### kemeny constant check
def check_kemeny(MFPT,peq):
    n = MFPT.shape[0]
    TEST = np.zeros(n)
    for i in range(n):
        for j in range(n):
            TEST[i] += MFPT[i,j] * peq[j]
    return TEST

In [24]:
TEST

array([9.73732926, 9.73732926, 9.73732926, 9.73732926, 9.73732926,
       9.73732926, 9.73732926, 9.73732926, 9.73732926, 9.73732926])

In [42]:
EDINA_MM = np.array([[0.4211  ,  0.4247  ,  0.1542],[0.2808 ,   0.2743 ,   0.4450],[0.9873  ,  0.3257,0.1018]])

In [67]:
print(EDINA_MM)

[[0.4211 0.4247 0.1542]
 [0.2808 0.2743 0.445 ]
 [0.9873 0.3257 0.1018]]


In [13]:
m = 5
for n in range(1,m):
    for i in range(n):
        print(i,n)

0 1
0 2
1 2
0 3
1 3
2 3
0 4
1 4
2 4
3 4


In [54]:
output = jjhunter(MM_dense)

9 [0.         0.08288717 0.03659691 0.19685122 0.17199895 0.
 0.16936991 0.00385645 0.09864392]
0.7602045272111011
8 [0.         0.14321247 0.00828696 0.         0.21463096 0.1989684
 0.         0.07762526]
0.6427240514436958
7 [0.13952965 0.21919002 0.00063551 0.21375478 0.15063478 0.1681201
 0.09515096]
0.9870158039749248
6 [0.         0.27743434 0.30003719 0.11121692 0.09069613 0.08407764]
0.8634622169865923
5 [0.08061996 0.15063523 0.14892663 0.22000706 0.22575299]
0.8259418713159876
4 [0.179356   0.30316423 0.02576148 0.07571393]
0.5839956344531279
3 [0.24921835 0.12681842 0.19776343]
0.5738002005646381
2 [0.07672171 0.22693517]
0.3036568808141076
1 [0.28031611]
0.28031610554135106
[0.         0.28031611 0.30365688 0.5738002  0.58399563 0.82594187
 0.86346222 0.9870158  0.64272405 0.76020453]
1.0 1.2949966774248944
[[0.0397605  0.05148971 0.11616455 0.07655607 0.11441339 0.09058892
  0.10757634 0.1052288  0.16159732 0.1366244 ]]


/tmp/ipykernel_2698/1067571557.py:35: DeprecationWarning: scipy.diag is deprecated and will be removed in SciPy 2.0.0, use numpy.diag instead
  DS = inv(diag(diag(PiS)))
/tmp/ipykernel_2698/1067571557.py:36: DeprecationWarning: scipy.diag is deprecated and will be removed in SciPy 2.0.0, use numpy.diag instead
  MS = (I - ZS + es @ diag(diag(ZS)))@DS


In [55]:
check_kemenyoutput

array([[25.15059076, 17.25299582,  9.01334739, 11.51042829, 10.15078713,
         9.38384467,  9.30058563,  9.49247117,  7.54696898,  9.5411217 ],
       [28.94854784, 19.42135543,  9.03885315, 11.69220102,  9.14655973,
        11.80455828,  8.6774626 ,  9.23612378,  7.10573953,  7.94098003],
       [30.39479644, 21.93912583,  8.60847788, 11.76596716, 10.33007737,
         8.75996874,  8.14127052,  8.34712912,  7.24766335,  8.86244786],
       [24.43040502, 21.21782977,  8.09505245, 13.06232223, 10.0793307 ,
         9.68311496,  8.01397603,  8.93090818,  7.2804021 ,  9.78995028],
       [26.41478356, 16.88185393,  9.72597362, 14.98501273,  8.74023536,
        11.58952951, 10.74742857,  8.85732762,  6.15928662,  7.47028449],
       [28.76869378, 20.14603433,  8.26169934, 13.08062603,  9.13623427,
        11.03887769, 10.24488412,  7.74371396,  7.4019737 ,  7.68423868],
       [31.91175524, 18.24184136,  6.69983102, 13.53033089, 10.44511379,
        11.00271077,  9.2957243 ,  9.40699803

In [51]:
print(I,"\n",P,"\n",PiS)

[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]] 
 [[0.         0.09829135 0.         0.         0.11939582 0.36263429
  0.41967854 0.         0.         0.        ]
 [0.         0.03476568 0.         0.         0.18816187 0.
  0.32200936 0.         0.45506308 0.        ]
 [0.         0.         0.         0.6289171  0.         0.
  0.3710829  0.         0.         0.        ]
 [0.         0.         0.         0.         0.36049888 0.
  0.18049068 0.15882029 0.08540947 0.21478068]
 [0.05983032 0.         0.         0.24673552 0.29777809 0.39565607
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.48296896 0.         0.15960806 0.35742297]
 [0.     

In [17]:
print(peq,sum(peq))

[0.17180743 0.10448532 0.11258229 0.15526671 0.03888526 0.13608462
 0.1192924  0.02166755 0.09429648 0.04563195] 1.0


In [76]:
check_kemeny(jjhunter(MM_dense),peq)

array([11.95679687, 11.54901641, 12.25145595, 11.12650215, 10.73061773,
       11.40776633, 11.68518255, 11.91371955, 10.6770274 , 12.50850439])

In [10]:
Markov_mfpt_calc_jjhunter(MM_dense.T)

[[ 31.10285408  23.02441596 -38.56907768  14.98261116  21.92636282
    7.05513902   5.74140855  27.02275556   3.83818891  -1.47187029]
 [ 10.15971251  21.33552648  -6.28299714   1.35534866  26.61660501
    4.52334593   4.02705787  26.95179024   3.31531652  11.32347308]
 [ 62.42255791  53.64869269  30.45216301  17.11350825  25.46771442
    8.48078099   4.76958098  26.65009551   0.66365234   7.67907685]
 [ -1.22830632  16.27506794 -32.55884535  10.0440507   22.13204437
    4.24543154   3.91933669  24.18587114   3.66618502   2.15425551]
 [  3.247167    16.5608888  -18.64756936   4.97755095  24.55914908
    3.26094832   1.85663865  26.42641635   2.08384062   8.41609015]
 [ -8.49566322  13.93382126 -47.17461022   6.04421773  15.13988353
    6.20336871   4.10144236  26.49567467   2.79739249  -0.88037954]
 [ -1.90472366  18.38149817 -31.13140922   5.00358446  18.92967177
    0.66806971   4.30686457  27.78719082   3.02436466   4.83747414]
 [ 14.01296938  16.47339529   1.14421667   7.0471957   

array([[ 31.10285408,  23.02441596, -38.56907768,  14.98261116,
         21.92636282,   7.05513902,   5.74140855,  27.02275556,
          3.83818891,  -1.47187029],
       [ 10.15971251,  21.33552648,  -6.28299714,   1.35534866,
         26.61660501,   4.52334593,   4.02705787,  26.95179024,
          3.31531652,  11.32347308],
       [ 62.42255791,  53.64869269,  30.45216301,  17.11350825,
         25.46771442,   8.48078099,   4.76958098,  26.65009551,
          0.66365234,   7.67907685],
       [ -1.22830632,  16.27506794, -32.55884535,  10.0440507 ,
         22.13204437,   4.24543154,   3.91933669,  24.18587114,
          3.66618502,   2.15425551],
       [  3.247167  ,  16.5608888 , -18.64756936,   4.97755095,
         24.55914908,   3.26094832,   1.85663865,  26.42641635,
          2.08384062,   8.41609015],
       [ -8.49566322,  13.93382126, -47.17461022,   6.04421773,
         15.13988353,   6.20336871,   4.10144236,  26.49567467,
          2.79739249,  -0.88037954],
       [ -

In [1]:
from util_old import TIME

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import time

from tqdm import tqdm

import openmm
from openmm import unit
from openmm.app.topology import Topology
from openmm.app.element import Element
import mdtraj
import csv

import config
from dham import *
from util_old import apply_fes, try_and_optim_M, apply_bias, update_bias, get_total_bias,try_and_optim_sparse_M, Markov_mfpt_calc, TIME
from scipy.sparse import csr_matrix

In [2]:
def propagate(simulation,
              prop_index, 
              pos_traj,   #this records the trajectory of the particle. in shape: [prop_index, sim_steps, 3]
              steps=config.propagation_step,
              dcdfreq=config.dcdfreq_mfpt,
              stepsize=config.stepsize,
              num_bins=config.num_bins,
              pbc=config.pbc,
              time_tag=None,
              top=None,
              reach=None
              ):
    """
    here we use the openmm context object to propagate the system.
    save the CV and append it into the CV_total.
    use the DHAM_it to process CV_total, get the partially observed Markov matrix from trajectory.
    return the current position, the CV_total, and the partially observed Markov matrix.
    """
    directory_paths = ["./trajectory/explore","./figs/explore","./params","./visited_states"]
    for i in directory_paths:
        if not os.path.exists(i):
            os.makedirs(i)
    file_handle = open(f"./trajectory/explore/{time_tag}_langevin_sim_explore_{prop_index}.dcd", 'bw')
    dcd_file = openmm.app.dcdfile.DCDFile(file_handle, top, dt = stepsize) #note top is no longer a global pararm, we need pass this.
    for _ in tqdm(range(int(steps/dcdfreq)), desc=f"Propagation {prop_index}"):
        simulation.integrator.step(dcdfreq)
        state = simulation.context.getState(getPositions=True, enforcePeriodicBox=pbc)
        dcd_file.writeModel(state.getPositions(asNumpy=True))
    file_handle.close()

    #save the top to pdb.
    with open(f"./trajectory/explore/{time_tag}_langevin_sim_explore_{prop_index}.pdb", 'w') as f:
        openmm.app.PDBFile.writeFile(simulation.topology, state.getPositions(), f)
    
    #we load the pdb and pass it to mdtraj_top
    mdtraj_top = mdtraj.load(f"./trajectory/explore/{time_tag}_langevin_sim_explore_{prop_index}.pdb")

    #use mdtraj to get the coordinate of the particle.
    traj = mdtraj.load_dcd(f"./trajectory/explore/{time_tag}_langevin_sim_explore_{prop_index}.dcd", top = mdtraj_top)#top = mdtraj.Topology.from_openmm(top)) #this will yield error because we using imaginary element X.
    coor = traj.xyz[:,0,:] #[all_frames,particle_index,xyz] # we grep the particle 0.

    #we digitize the x, y coordinate into meshgrid (0, 2pi, num_bins)
    x = np.linspace(0, 2*np.pi, num_bins) #hardcoded.

    #we digitize the coor into the meshgrid.
    coor_x = coor.squeeze()[:,:1] #we only take the xcoordinate.
    #we test.
    if False: 
        plt.figure()
        plt.plot(coor_x)
        plt.xlim([0, 2*np.pi])
        plt.savefig("./test.png")
        plt.close()
    #we append the coor_xy_digitized into the pos_traj.
    pos_traj[prop_index,:] = coor_x.squeeze()

    #we take all previous digitized x and feed it into DHAM.
    coor_x_total = pos_traj[:prop_index+1,:].squeeze() #note this is in coordinate space np.linspace(0, 2*np.pi, num_bins)
    print("coor_x_total shape: ", coor_x_total.shape)
    #we now reshape it to [cur_propagation+1, num_bins, 1]
    coor_x_total = coor_x_total.reshape(prop_index+1, -1, 1)
    print("coor_x_total shape: ", coor_x_total.shape)

    #here we load all the gaussian_params from previous propagations.
    #size of gaussian_params: [num_propagation, num_gaussian, 3] (a,b,c),
    # note for 2D this would be [num_propagation, num_gaussian, 5] (a,bx,by,cx,cy)
    gaussian_params = np.zeros([prop_index+1, config.num_gaussian, 3])
    for i in range(prop_index+1):
        gaussian_params[i,:,:] = np.loadtxt(f"./params/{time_tag}_gaussian_fes_param_{i}.txt").reshape(-1,3)
        print(f"gaussian_params for propagation {i} loaded.")
    print(gaussian_params)

    #here we use the DHAM.
    F_M, MM = DHAM_it(coor_x_total, 
                      gaussian_params, 
                      T=300, 
                      lagtime=1, 
                      num_bins=num_bins, 
                      time_tag=time_tag, 
                      prop_index=prop_index)
    cur_pos = coor_x_total[-1] #the current position of the particle, in ravelled 1D form.
    
    #determine if the particle has reached the target state.
    end_state_xyz = config.end_state.value_in_unit_system(openmm.unit.md_unit_system)[0]
    end_state_x = end_state_xyz[:1]
    for index_d, d in enumerate(coor_x):
        #if the distance of current pos is the config.target_state, we set reach to index_d.
        target_distance = np.linalg.norm(d - end_state_x)
        if target_distance < 0.1:
            reach = index_d * config.dcdfreq_mfpt

    return cur_pos, pos_traj, MM, reach, F_M
def get_working_MM(M):
    zero_rows = np.where(~M.any(axis=1))[0]
    zero_cols = np.where(~M.any(axis=0))[0]

    keep_indices = np.setdiff1d(range(M.shape[0]), np.union1d(zero_rows, zero_cols))
    M_work = M[np.ix_(keep_indices, keep_indices)]
    return M_work, keep_indices

def find_closest_index(working_indices, final_index, N):
    """
    returns the farest index in 1D.

    here we find the closest state to the final state.
    first we unravel all the index to 2D.
    then we use the lowest RMSD distance to find the closest state.
    then we ravel it back to 1D.
    note: for now we only find the first-encounted closest state.
          we can create a list of all the closest states, and then choose random one.
    """
    def rmsd_dist(a, b):
        return np.sqrt(np.sum((a-b)**2))
    working_x, working_y = np.unravel_index(working_indices, (N,N), order='C')
    working_states = np.stack((working_x, working_y), axis=1)
    final_state = np.unravel_index(final_index, (N,N), order='C')
    closest_state = working_states[0]
    for i in range(len(working_states)):
        if rmsd_dist(working_states[i], final_state) < rmsd_dist(closest_state, final_state):
            closest_state = working_states[i]
        
    closest_index = np.ravel_multi_index(closest_state, (N,N), order='C')
    return closest_index

def DHAM_it(CV, gaussian_params, T=300, lagtime=2, num_bins=150, prop_index=0, time_tag=None):
    """
    intput:
    CV: the collective variable we are interested in. now it's 2d.
    gaussian_params: the parameters of bias potential. (in our case the 10-gaussian params)
     format: (a,bx, by,cx,cy)
    T: temperature 300

    output:
    the Markov Matrix
    Free energy surface probed by DHAM.
    """
    d = DHAM(gaussian_params, num_bins=num_bins)
    d.setup(CV, T, prop_index=prop_index, time_tag=time_tag)

    d.lagtime = lagtime
    d.num_bins = num_bins #num of bins, arbitrary.
    results = d.run(biased = True, plot=True)
    return results

def random_initial_bias(initial_position, num_gaussians):
    #returns random a,b,c for 10 gaussian functions. around the initial position
    # initial position is in Anstrom
    initial_position = initial_position.value_in_unit_system(openmm.unit.md_unit_system)[0] #this is in nm
    rng = np.random.default_rng()
    #a = np.ones(10)
    a = np.ones(num_gaussians) * 0.01 * 4.184 #convert to kJ/mol
    b = rng.uniform(initial_position[0]-0.5, initial_position[0]+0.5, num_gaussians)
    c = rng.uniform(0, 2*np.pi, num_gaussians)
    return np.concatenate((a,b,c), axis=None)


In [3]:
time_class = TIME(sections=5)
if __name__ == "__main__":
    elem = Element(0, "X", "X", 1.0)
    top = Topology()
    top.addChain()
    top.addResidue("xxx", top._chains[0])
    top.addAtom("X", elem, top._chains[0]._residues[0])
    mass = 12.0 * unit.amu
    for i_sim in range(1):
    #def simulate_once():
        print("system initializing")
        #print out all the config.
        print("config: ", config.__dict__)
        
        time_tag = time.strftime("%Y%m%d-%H%M%S")

        #print current time tag.
        print("time_tag: ", time_tag)

        system = openmm.System() #we initialize the system every
        system.addParticle(mass)
        #gaussian_param = np.loadtxt("./params/gaussian_fes_param.txt")
        system, fes = apply_fes(system = system, 
                                particle_idx=0, 
                                gaussian_param = None, 
                                pbc = config.pbc, 
                                amp = config.amp, 
                                name = "FES",
                                mode=config.fes_mode, 
                                plot = True)
        y_pot = openmm.CustomExternalForce("1e3 * y^2") # very large force constant in y
        y_pot.addParticle(0)
        z_pot = openmm.CustomExternalForce("1e3 * z^2") # very large force constant in z
        z_pot.addParticle(0)
        system.addForce(z_pot) #on z, large barrier
        system.addForce(y_pot) #on y, large barrier

        #pbc section
        if config.pbc:
            a = unit.Quantity((2*np.pi*unit.nanometers, 0*unit.nanometers, 0*unit.nanometers))
            b = unit.Quantity((0*unit.nanometers, 2*np.pi*unit.nanometers, 0*unit.nanometers))
            c = unit.Quantity((0*unit.nanometers, 0*unit.nanometers, 1*unit.nanometers)) # atom not moving in z so we set it to 1 nm
            system.setDefaultPeriodicBoxVectors(a,b,c)

        #integrator
        integrator = openmm.LangevinIntegrator(300*unit.kelvin, 
                                            1.0/unit.picoseconds, 
                                            0.002*unit.picoseconds)

        num_propagation = int(config.sim_steps/config.propagation_step)
        frame_per_propagation = int(config.propagation_step/config.dcdfreq)
        #this stores the digitized, ravelled, x, y coordinates of the particle, for every propagation.
        pos_traj = np.zeros([num_propagation, frame_per_propagation]) #shape: [num_propagation, frame_per_propagation]

        x = np.linspace(0, 2*np.pi, config.num_bins)

        #we start propagation.
        #note num_propagation = config.sim_steps/config.propagation_step
        reach = None
        i_prop = 0
        #for i_prop in range(num_propagation):
        while reach is None:
            if i_prop >= num_propagation:
                print("propagation number exceeds num_propagation, break")
                break
            if i_prop == 0:
                print("propagation 0 starting")
                gaussian_params = random_initial_bias(initial_position = config.start_state, num_gaussians = config.num_gaussian)

                np.savetxt(f"./params/{time_tag}_gaussian_fes_param_{i_prop}.txt", gaussian_params)
                
                #we apply the initial gaussian bias (v small) to the system
                system = apply_bias(system = system, particle_idx=0, gaussian_param = gaussian_params, pbc = config.pbc, name = "BIAS", num_gaussians = config.num_gaussian)

                #create simulation object, this create a context object automatically.
                # when we need to pass a context object, we can pass simulation instead.
                simulation = openmm.app.Simulation(top, system, integrator, config.platform)
                simulation.context.setPositions(config.start_state)
                simulation.context.setVelocitiesToTemperature(300*unit.kelvin)

                simulation.minimizeEnergy()
                if config.pbc:
                    simulation.context.setPeriodicBoxVectors(a,b,c)

                #now we propagate the system, i.e. run the langevin simulation.
                cur_pos, pos_traj, MM, reach, F_M = propagate(simulation = simulation,
                                                                    prop_index = i_prop,
                                                                    pos_traj = pos_traj,
                                                                    steps=config.propagation_step,
                                                                    dcdfreq=config.dcdfreq,
                                                                    stepsize=config.stepsize,
                                                                    num_bins=config.num_bins,
                                                                    pbc=config.pbc,
                                                                    time_tag = time_tag,
                                                                    top=top,
                                                                    reach=reach
                                                                    )

                #working_MM, working_indices = get_working_MM(MM) no longer needed as sparse matrix data structures are more efficient
                MM_sparse = config.sparse_mat[0](MM)
                working_indices = np.intersect1d(np.unique(MM_sparse.row),np.unique(MM_sparse.col))
                MM_sparse = MM_sparse.tocsr()
                final_coor = config.end_state.value_in_unit_system(openmm.unit.md_unit_system)[0][:1]
                final_index = np.digitize(final_coor, x)
                closest_index = working_indices[np.argmin(np.abs(working_indices - final_index))] #find the closest index in working_indices to final_index.
                i_prop += 1
            else:

                print(f"propagation number {i_prop} starting")

                #find the most visited state in last propagation.
                last_traj = pos_traj[i_prop-1,:]
                last_traj_index = np.digitize(last_traj, x).astype(np.int64)
                most_visited_state = np.argmax(np.bincount(last_traj_index)) #this is in digitized

                ##### time
                #gaussian_params 
                gaussian_params,time_class= try_and_optim_sparse_M(MM_sparse,working_indices,time_class,num_gaussian = config.num_gaussian, start_index = most_visited_state,end_index = closest_index,plot = False)
                # gaussian_params = try_and_optim_M(working_MM,
                #                                 working_indices = working_indices,
                #                                 num_gaussian = config.num_gaussian,
                #                                 start_index = most_visited_state,
                #                                 end_index = closest_index,
                #                                 plot = False,
                #                                 )
                if True:
                    #here we calculate the total bias given the optimized gaussian_params
                    total_bias = get_total_bias(x, gaussian_params, num_gaussians=config.num_gaussian) * 4.184 #convert to kcal/mol
                    plt.figure()
                    plt.plot(x, total_bias, label="total bias applied")
                    #plt.savefig(f"./figs/explore/{time_tag}_total_bias_{i_prop}.png")
                    #plt.close()
                    
                    #here we plot the reconstructed fes from MM.
                    # we also plot the most_visited_state and closest_index.
                    #plt.figure()
                    plt.plot(x, F_M*4.184, label="DHAM fes")
                    plt.plot(x[most_visited_state], F_M[most_visited_state]*4.184, marker='o', markersize=3, color="blue", label = "most visited state (local start)")
                    plt.plot(x[closest_index], F_M[closest_index]*4.184, marker='o', markersize=3, color="red", label = "closest state (local target)")
                    #plt.legend()
                    #plt.savefig(f"./figs/explore/{time_tag}_reconstructed_fes_{i_prop}.png")
                    #plt.close()

                    #we plot here to check the original fes, total_bias and trajectory.
                
                    #we add the total bias to the fes.
                    #fes += total_bias_big
                    #plt.figure()
                    plt.plot(x, fes, label="original fes")
                    plt.xlabel("x-coor position (nm)")
                    #plt.xlim([-1, 2*np.pi+1])
                    #plt.ylim([-1, 2*np.pi+1])
                    plt.ylabel("fes (kJ/mol)")
                    plt.title("FES mode = multiwell, pbc=False")
                    
                    #additionally we plot the trajectory.
                    # first we process the pos_traj into x, y coordinate.
                    # we plot all, but highlight the last prop_step points with higher alpha.

                    history_traj = pos_traj[:i_prop, :].squeeze() #note this is only the x coor.
                    recent_traj = pos_traj[i_prop:, :].squeeze()
                    #here we digitize this so we can plot it to fes.
                    history_traj = np.digitize(history_traj, x)
                    recent_traj = np.digitize(recent_traj, x)

                    plt.scatter(x[history_traj], fes[history_traj], s=3.5, alpha=0.3, c='grey')
                    plt.scatter(x[recent_traj], fes[recent_traj], s=3.5, alpha=0.8, c='black')
                    plt.legend(loc='upper left')
                    plt.savefig(f"./figs/explore/{time_tag}_fes_traj_{i_prop}.png")
                    plt.close()


                #save the gaussian_params
                np.savetxt(f"./params/{time_tag}_gaussian_fes_param_{i_prop}.txt", gaussian_params)

                #apply the gaussian_params to openmm system.
                simulation = update_bias(simulation = simulation,
                                        gaussian_param = gaussian_params,
                                        name = "BIAS",
                                        num_gaussians=config.num_gaussian,
                                        )
                
                #we propagate system again
                cur_pos, pos_traj, MM, reach, F_M = propagate(simulation = simulation,
                                                                    gaussian_params = gaussian_params,
                                                                    prop_index = i_prop,
                                                                    pos_traj = pos_traj,
                                                                    steps=config.propagation_step,
                                                                    dcdfreq=config.dcdfreq,
                                                                    stepsize=config.stepsize,
                                                                    num_bins=config.num_bins,
                                                                    pbc=config.pbc,
                                                                    time_tag = time_tag,
                                                                    top=top,
                                                                    reach=reach
                                                                    )
                #update working_MM and working_indices
                MM_sparse = config.sparse_mat[0](MM)
                working_indices = np.intersect1d(np.unique(MM_sparse.row),np.unique(MM_sparse.col))
                MM_sparse = MM_sparse.tocsr()
                #update closest_index
                closest_index = working_indices[np.argmin(np.abs(working_indices - final_index))] #find the closest index in working_indices to final_index.
                
                i_prop += 1

        #we have reached target state, thus we record the steps used.
        total_steps = i_prop * config.propagation_step + reach * config.dcdfreq
        print("total steps used: ", total_steps)

        with open("./total_steps_mfpt.csv", "a") as f:
            writer = csv.writer(f)
            writer.writerow([total_steps])

        #save the pos_traj
        np.savetxt(f"./visited_states/{time_tag}_pos_traj.txt", pos_traj)

    """from multiprocessing import Pool
    
    multi_process_result = []
    for _ in range(config.num_sim//config.NUMBER_OF_PROCESSES):
        with Pool(config.NUMBER_OF_PROCESSES) as p:
            multi_process_result.extend(p.map(simulate_once, range(config.NUMBER_OF_PROCESSES)))
"""
print("all done")


system initializing
config:  {'__name__': 'config', '__doc__': None, '__package__': '', '__loader__': <_frozen_importlib_external.SourceFileLoader object at 0x7fcad844ead0>, '__spec__': ModuleSpec(name='config', loader=<_frozen_importlib_external.SourceFileLoader object at 0x7fcad844ead0>, origin='/home/gsaba/MarkovMastersProject/RL_MFPT/1D_langevin_sim/config.py'), '__file__': '/home/gsaba/MarkovMastersProject/RL_MFPT/1D_langevin_sim/config.py', '__cached__': '/home/gsaba/MarkovMastersProject/RL_MFPT/1D_langevin_sim/__pycache__/config.cpython-311.pyc', '__builtins__': {'__name__': 'builtins', '__doc__': "Built-in functions, types, exceptions, and other objects.\n\nThis module provides direct access to all 'built-in'\nidentifiers of Python; for example, builtins.len is\nthe full name for the built-in function len().\n\nThis module is not normally accessed explicitly by most\napplications, but can be useful in modules that provide\nobjects with the same name as a built-in value, but in\

Propagation 0: 100%|██████████| 50/50 [00:00<00:00, 225.35it/s]
/home/gsaba/MarkovMastersProject/RL_MFPT/1D_langevin_sim/util.py:114: RuntimeWarning: divide by zero encountered in log
  F = -kT * np.log(peq) #add a small number to avoid log(0)) # + 1e-16


coor_x_total shape:  (50,)
coor_x_total shape:  (1, 50, 1)
gaussian_params for propagation 0 loaded.
[[[0.04184    0.04184    0.04184   ]
  [0.04184    0.04184    0.04184   ]
  [0.04184    0.04184    0.04184   ]
  [0.04184    0.04184    0.04184   ]
  [0.04184    0.04184    0.04184   ]
  [0.04184    0.04184    0.04184   ]
  [0.04184    0.04184    1.56308597]
  [2.34751419 1.78614938 1.92661616]
  [2.36003352 2.29192365 2.14598384]
  [1.51887859 1.55436706 1.60366025]
  [1.78110518 2.44146902 1.7197599 ]
  [2.17136094 1.52273923 2.25431501]
  [2.3850129  1.50143909 2.26480035]
  [1.69556617 3.36577087 4.50117661]
  [0.31255362 4.94708736 1.91117803]
  [3.22245053 5.30822555 4.24228276]
  [3.50871852 4.39007057 6.04787564]
  [0.66892821 4.5670026  5.52091337]
  [3.20295515 3.74254357 5.98400451]
  [2.17733718 5.72788345 4.34620286]]]
sum of peq in dham reconstruction:  1.0000000000000002
propagation number 1 starting
optimizing to get g_param from start state: 0 to end state: 39 in workin

/home/gsaba/MarkovMastersProject/RL_MFPT/1D_langevin_sim/util_old.py:124: RuntimeWarning: invalid value encountered in log
  F = -kT * np.log(peq)
/home/gsaba/miniconda3/envs/biophys_env/lib/python3.11/site-packages/scipy/sparse/linalg/_dsolve/linsolve.py:412: SparseEfficiencyWarning: splu converted its input to CSC format
  warn('splu converted its input to CSC format', SparseEfficiencyWarning)
/home/gsaba/miniconda3/envs/biophys_env/lib/python3.11/site-packages/scipy/sparse/linalg/_dsolve/linsolve.py:302: SparseEfficiencyWarning: spsolve is more efficient when sparse b is in the CSC matrix format
  warn('spsolve is more efficient when sparse b '


random try: 0 mfpt: 154.16286309398427
Performing Kemeny constant check...
the min/max of the Kemeny constant is: 1089.2711897154804 1090.3097569502174
random try: 100 mfpt: 230.51579511964292
Performing Kemeny constant check...
the min/max of the Kemeny constant is: 1088.9197022292399 1090.3097569502177
random try: 200 mfpt: 209.98945617424602
Performing Kemeny constant check...
the min/max of the Kemeny constant is: 1089.0482192529435 1090.3097569502174
random try: 300 mfpt: 174.96365083255407
Performing Kemeny constant check...
the min/max of the Kemeny constant is: 1089.2846074512966 1090.3097569502177


KeyboardInterrupt: 

In [4]:
TIME1.best

490.7893033190003

In [4]:
gaussian_params

array([0.04184   , 0.04184   , 0.04184   , 0.04184   , 0.04184   ,
       0.04184   , 0.04184   , 0.04184   , 0.04184   , 0.04184   ,
       0.04184   , 0.04184   , 0.04184   , 0.04184   , 0.04184   ,
       0.04184   , 0.04184   , 0.04184   , 0.04184   , 0.04184   ,
       1.93006573, 1.66835713, 1.8416719 , 2.31597601, 1.55843228,
       2.23336044, 2.38793466, 1.86535852, 2.13711174, 1.71808267,
       2.40904137, 1.56084391, 1.71245887, 1.92344673, 2.18928872,
       2.10560723, 1.88467782, 1.80386112, 2.47301529, 2.21037727,
       4.49965069, 6.07742285, 2.58236963, 3.4926577 , 3.61444423,
       4.92156731, 4.2370975 , 2.86333773, 3.75117948, 5.03213536,
       1.67815781, 0.64870149, 0.20405598, 1.11479101, 6.11502661,
       3.01996509, 1.58868883, 3.85506198, 5.62998665, 3.43101277])

In [5]:
i_prop

100

In [21]:
def Markov_mfpt_calc_sparse(peq, M):
    N = M.shape[0]
    onevec = np.ones(N)
    Idn = eye(N)  # Sparse identity matrix
    print("peq: ",peq)
    print("peq reshaped (-1,1)",peq.reshape(-1,1))
    A = config.sparse_mat[1](((peq.reshape(-1, 1)) @ onevec.T).T)
    print(M.data)
    print(Idn + A - M)
    Qinv = linalg.inv(Idn + A - M)  # Sparse matrix inversion

    mfpt = np.zeros((N, N))
    for j in range(N):
        for i in range(N):
            term1 = Qinv[j, j] - Qinv[i, j] + Idn[i, j]
            if peq[j] * term1 == 0:
                mfpt[i, j] = 1e12  # Use a large number instead of explicit value
            else:
                mfpt[i, j] = 1/peq[j] * term1
    return mfpt

In [33]:
from scipy.linalg import inv

In [32]:
%%time
linalg.inv((Idn + A - M).astype('float64'))

CPU times: user 10.8 ms, sys: 0 ns, total: 10.8 ms
Wall time: 10.4 ms


/home/gsaba/miniconda3/envs/biophys_env/lib/python3.11/site-packages/scipy/sparse/linalg/_dsolve/linsolve.py:412: SparseEfficiencyWarning: splu converted its input to CSC format
  warn('splu converted its input to CSC format', SparseEfficiencyWarning)
/home/gsaba/miniconda3/envs/biophys_env/lib/python3.11/site-packages/scipy/sparse/linalg/_dsolve/linsolve.py:302: SparseEfficiencyWarning: spsolve is more efficient when sparse b is in the CSC matrix format
  warn('spsolve is more efficient when sparse b '


<100x100 sparse matrix of type '<class 'numpy.float64'>'
	with 2476 stored elements in Compressed Sparse Row format>

In [35]:
Q_dense = (Idn + A - M).astype('float64').toarray()

In [36]:
%%time
inv(Q_dense)

CPU times: user 853 ms, sys: 360 ms, total: 1.21 s
Wall time: 82.2 ms


array([[ 1., -0.,  0., ..., -0.,  0., -0.],
       [ 0.,  1.,  0., ..., -0.,  0., -0.],
       [ 0.,  0.,  1., ..., -0.,  0., -0.],
       ...,
       [ 0.,  0.,  0., ...,  1.,  0., -0.],
       [ 0.,  0.,  0., ...,  0.,  1., -0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  1.]])

In [24]:
Idn.shape

(100, 100)

In [29]:
peq = peq[:100]

In [26]:
A.shape

(100, 108)

In [36]:
mfpt_optimized_sparse.toarray()[40]

array([1.00000000e+12, 1.00000000e+12, 1.00000000e+12, 1.00000000e+12,
       1.00000000e+12, 1.00000000e+12, 1.00000000e+12, 1.00000000e+12,
       1.00000000e+12, 1.00000000e+12, 1.00000000e+12, 1.00000000e+12,
       1.00000000e+12, 1.00000000e+12, 1.00000000e+12, 1.00000000e+12,
       1.00000000e+12, 1.00000000e+12, 1.00000000e+12, 1.00000000e+12,
       1.00000000e+12, 1.00000000e+12, 1.00000000e+12, 1.00000000e+12,
       1.00000000e+12, 1.00000000e+12, 1.00000000e+12, 1.00000000e+12,
       3.62188324e+01, 3.46966066e+01, 4.48058018e+01, 5.47956375e+01,
       4.44540330e+01, 4.67232899e+01, 4.88219649e+01, 3.17381696e+01,
       2.44102012e+01, 1.46883066e+01, 6.63215695e+00, 1.08430610e+01,
       3.81758771e+01, 2.74186962e+01, 2.31730459e+01, 1.00000000e+12,
       1.00000000e+12, 1.00000000e+12, 1.00000000e+12, 1.00000000e+12,
       1.00000000e+12, 1.00000000e+12, 1.00000000e+12, 1.00000000e+12,
       1.00000000e+12, 1.00000000e+12, 1.00000000e+12, 1.00000000e+12,
      

In [11]:
from scipy.sparse import diags, eye
from scipy.sparse import linalg,diags

In [12]:
Markov_mfpt_calc_sparse(peq, bias_mat)

peq:  [0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.06067021 0.06146722
 0.0622529  0.0630296  0.06380288 0.06458124 0.06537591 0.06620055
 0.06707084 0.06800414 0.06901899 0.07013462 0.07137035 0.0727449
 0.07427566 0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.  

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 100 is different from 1)

In [4]:
MM_sparse.data

array([1.        , 0.50259762, 0.49740238, 1.        , 0.20071176,
       0.40029615, 0.19969948, 0.19929261, 0.14321546, 0.57157787,
       0.28520667, 0.08391362, 0.1669808 , 0.49992177, 0.16627136,
       0.08291246, 0.20057046, 0.4002521 , 0.39917744, 0.16790454,
       0.16753251, 0.16708269, 0.16652821, 0.16586175, 0.16509031,
       0.50306022, 0.49693978, 0.67048231, 0.32951769, 0.50116548,
       0.49883452, 0.33642846, 0.66357154], dtype=float128)

In [6]:
print(csr_matrix(np.zeros((5,5))).data)

[]


In [4]:
print(MM_sparse)

  (29, 29)	0.3332803572911920194
  (29, 30)	0.6667196427088079806
  (30, 29)	0.33335530786434554548
  (30, 31)	0.6666446921356544545
  (31, 29)	0.25042258794774740468
  (31, 30)	0.2504822962261473101
  (31, 33)	0.24980056624349338352
  (31, 34)	0.24929454958261190167
  (32, 31)	1.0
  (33, 33)	0.33378409375424774843
  (33, 34)	0.66621590624575225157
  (34, 32)	0.2011740994518226312
  (34, 33)	0.20087721366960539249
  (34, 35)	0.19995818209381306999
  (34, 36)	0.19934691037578208208
  (34, 37)	0.19864359440897682424
  (35, 34)	0.40085887530288522926
  (35, 35)	0.39983484787837184025
  (35, 36)	0.19930627681874293048
  (36, 34)	0.25130545563339104994
  (36, 35)	0.2506634748906570818
  (36, 37)	0.49803106947595186826
  (37, 35)	0.16797158512862434545
  (37, 36)	0.33491619273278013137
  (37, 38)	0.33241171407365481302
  (37, 40)	0.16470050806494071014
  (38, 39)	1.0
  (39, 37)	0.3354526787922488724
  (39, 39)	0.49899891592191072395
  (39, 40)	0.16554840528584040366
  (40, 37)	0.252809188566